# **IMPORTS AND LOADING HELPER FILES**

I run this notebook on Google Colab. Please upload the colabTest.zip file before running the first cell.

## **Imports**

In [1]:
!unzip colabTest.zip
import pandas as pd
from google.colab import files
!apt install sqlite3
df = pd.read_csv('./Final_50_Sample.csv')
from os.path import join
import json
import random
import numpy as np
from utils import *
import os
import json
import time


import pandas as pd
from utils import *
from itertools import compress

import os
import json
map_func = {'sum': np.sum, 'avg': np.mean, 'max': np.max, 'count': len}

!git clone https://github.com/openai/openai-python.git
!cd openai-python
!pip install aiohttp
!python setup.py install
!pip install tenacity
import numpy as np
!pip install openai
import openai
import os
import json
import time
import openai
from tqdm import tqdm
openai.api_key = "YOUR OPENAI KEY"
from math import ceil
from time import sleep

import subprocess
import duckdb
import subprocess
import os
!pip install rouge
from rouge import Rouge

Archive:  colabTest.zip
  inflating: Final_50_Sample.csv     
  inflating: utils.py                
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Cloning into 'openai-python'...
remote: Enumerating objects: 1576, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 1576 (delta 396), reused 353 (delta 348), pack-reused 1118
Receiving objects: 100% (1576/1576), 1.88 MiB | 5.57 MiB/s, done.
Resolving deltas: 100% (943/943), done.
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00


## **QueryTree**

In [2]:
box_head1 = '┌───────────────────────────┐'

box_head2 = '┌─────────────┴─────────────┐'

box_end1  = '└─────────────┬─────────────┘'

box_end2  = '└───────────────────────────┘'

class Node:
    def __init__(self):
        self.text, self.l, self.r, self.op, self.args = None, None, None, None, None
        self.questions, self.unfiltered_answers, self.answers = [], [], []

def parse_query_tree(s):
    root = Node()
    p=root
    for ss in s:
        if ss ==box_head1 or ss== box_head2:
          seach_text=True
          text=[]
        elif  box_end1 in ss or box_end2 in ss:
          search_text=False
          p.text = text
          p.op = p.text[0]
          p.args = p.text[1:]
          p.l=Node()
          p=p.l
        else:
          t=ss.replace('|','').replace('─','').replace('│','').replace('└┬┘','').replace('└┘','').strip()#.replace(' ','')
          if t:
              text.append(t)
              if ')' in text[-1] and '(' not in text[-1]: text = text[:-2]+[" ".join(text[-2:])]
    # remove projection after aggregation
    if root.op=='PROJECTION' and root.l and root.l.op=='AGGREGATE' and root.args == root.l.args: root=root.l
    return root


def parse_query_tree(s):
    print(s)
    root = Node()  # Create the root node
    p = root  # Set the current node as the root

    for ss in s:
        if ss == box_head1 or ss == box_head2:  # Check if ss matches box_head1 or box_head2
            search_text = True  # Set a flag to indicate searching for text
            text = []  # Initialize an empty list to store text
        elif box_end1 in ss or box_end2 in ss:  # Check if box_end1 or box_end2 is present in ss
            search_text = False  # Set the flag to indicate the end of searching for text
            p.text = text  # Assign the collected text to the current node
            p.op = p.text[0]  # Assign the operation to the current node
            p.args = p.text[1:]  # Assign the arguments to the current node
            p.l = Node()  # Create a new node and assign it as the left child of the current node
            p = p.l  # Update the current node to the left child
        else:
            t = ss.replace('|','').replace('─','').replace('│','').replace('└┬┘','').replace('└┘','').strip()
            # Clean up the string ss and remove unnecessary characters

            if t:
                text.append(t)  # Append the cleaned text to the list

                if ')' in text[-1] and '(' not in text[-1]:
                    # Check if the last element in text contains a closing parenthesis without an opening parenthesis
                    text = text[:-2] + [" ".join(text[-2:])]  # Merge the last two elements in text with a space

    # Remove projection after aggregation
    if root.op == 'PROJECTION' and root.l and root.l.op == 'AGGREGATE' and root.args == root.l.args:
        root = root.l  # Update the root to the left child

    return root  # Return the root node


def print_tree(p):
    if p and p.text:
      print_tree(p.l)
      print_tree(p.r)

def get_tree_elements(p, a):
    if p and p.text:
        a.append(p.text)

        # Recursively call get_tree_elements on p's left and right child
        get_tree_elements(p.l, a)
        get_tree_elements(p.r, a)

    return a


def get_snippet(p, tree_nodes, questions, answers, unfiltered_answers):
    """
    Recursively traverses a tree and extracts relevant data into separate lists.

    Args:
        p (Node): Current node in the tree
        tree_nodes (list): List to store adjusted nodes
        questions (list): List to store questions
        answers (list): List to store answers
        unfiltered_answers (list): List to store unfiltered answers

    Returns:
        tuple: Lists containing adjusted nodes, questions, answers, and unfiltered answers
    """
    if p and p.text:
        # Traverse the left subtree
        get_snippet(p.l, tree_nodes, questions, answers, unfiltered_answers)
        # Traverse the right subtree
        get_snippet(p.r, tree_nodes, questions, answers, unfiltered_answers)

        # Append data from the current node to respective lists
        tree_nodes.append(p.adjusted_nodes)
        questions.append(p.questions)
        answers.append(p.answers)
        unfiltered_answers.append(p.unfiltered_answers)

    # Return the collected data as tuples
    return tree_nodes, questions, answers, unfiltered_answers

## **ChatGPT Calls**

In [3]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    wait_fixed
)
import openai
@retry(wait=wait_fixed(61), stop=stop_after_attempt(3))
def completion_with_backoff_chat(**kwargs):
    #print(f"Value of kwaargs: {kwargs}")
    #print(f"{type(**kwargs)}")
    #print('=======Calling API=======')
    #sleep(21)
    return openai.ChatCompletion.create(**kwargs)

def construct_chat_dict(role,content):
  return {"role":role,"content":content}

def construct_message_dict(instruction, question_answers):
    messages = []

    # Add system instruction to the list of messages
    messages.append(construct_chat_dict("system", instruction))

    # Iterate through each question-answer pair
    for question, answer in question_answers:
        # Add user's question to the list of messages
        messages.append(construct_chat_dict("user", question))
        # Add assistant's answer to the list of messages
        messages.append(construct_chat_dict("assistant", answer))

    return messages

def add_more_seq_scan(temp_unfilt_ans, model_arch, temp_questions, old_pr, old_ans, max_tries=0, increase_threshold=5, verbose=True):
    i = 0
    final_ans = old_ans

    while i < max_tries:
        if verbose:
            print('#', i + 1)

        # Create the prompt for the GPT-3 API
        pr = old_pr + [construct_chat_dict("assistant", old_ans)] + [construct_chat_dict("user", 'Give me more.')]
        messages = pr
        temp_questions.append("Give me more.")

        # Run GPT-3 to generate a response
        max_len = 400
        response = completion_with_backoff_chat(model=model_arch, messages=messages, temperature=0, max_tokens=max_len)

        ans = response['choices'][0]['message']['content']

        if verbose:
            print(ans)
        temp_unfilt_ans.append(ans)

        final_ans += ans

        len1 = len(set(old_ans.split(',')))
        len2 = len(set(old_ans.split(',') + ans.split(',')))

        old_ans = ans
        old_pr = pr

        if (len2 - len1) < increase_threshold:
            break

        i += 1

    return final_ans


def answer_batch_questions_chat(model_questions,pr,label,cache_fn,model_arch,max_len,verbose=False):
    cache_fn=label+'_cache.json'
    mode = 'r' if cache_fn in os.listdir('.') else 'w'
    cache=json.load(open(cache_fn,'r')) if mode=='r' else dict()
    ans=[]
    batch_pr = pr[:]
    batch_mq = model_questions[:]
    batch_ans = [None]*len(batch_pr)
    to_fetch_indices=[]

    for ind,pro in enumerate(batch_pr):
        pro_key = json.dumps(pro) if isinstance(pro,list) else pro
        if pro_key in cache:
            #print('Already in cache: ',batch_mq[ind],'\n')
            batch_ans[ind]=cache[pro_key]
        else:
            to_fetch_indices.append(ind)
    #print('INITIAL BATCH_ANS:',batch_ans)
    if verbose:
        print(f'In Cache: {len(batch_ans)-len(to_fetch_indices)}/{len(batch_ans)}')
    if to_fetch_indices:
        batch_pr_to_fetch = [batch_pr[tfi] for tfi in to_fetch_indices]
        batch_mq_to_fetch = [batch_mq[tfi] for tfi in to_fetch_indices]

        for i in range(len(batch_pr_to_fetch)):
            #print(batch_pr_to_fetch[i])
            response = completion_with_backoff_chat(model=model_arch, messages=batch_pr_to_fetch[i], temperature=0,max_tokens= max_len)

            if 'choices' in response:
                #print('RESPONSE:',response)
                batch_ans_fetched = response['choices'][0]['message']['content']

                #print('BATCH ANS FETCHED:',batch_ans_fetched)


                bpr = batch_pr_to_fetch[i]
                bmq= batch_mq_to_fetch[i]
                bans= batch_ans_fetched
                bpr_key = json.dumps(bpr) if isinstance(bpr,list) else bpr
                cache[bpr_key]=bans
                print('Added to cache:',bmq,'\n\n')
                print(f'Len of cache: {len(cache)}')
                json.dump(cache,open(cache_fn,"w"),indent=2)
            else: batch_ans_fetched=[]

            batch_ans[to_fetch_indices[i]] =batch_ans_fetched

        ans.extend(batch_ans)
    return batch_ans


def compute_node(node,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=False):

    cache_fn=label+'_cache.json'
    mode = 'r' if cache_fn in os.listdir('.') else 'w'
    if verbose: print('Mode: ',mode)
    cache=json.load(open(cache_fn,'r')) if mode=='r' else dict()
    #print(f'Len of cache: {len(cache)}')
    status = 'FINISHED'

    if node.op=='JOIN':
        #pr= instr+few_shots+""
        pr=construct_message_dict(instr,few_shots)




        left_questions = [node.key_left.replace('!!x!!',x) for x in node.l.answers[-1]]
        print('left questions',left_questions)


        #lpr = [pr + inst_funct(x) for x in left_questions]
        lpr = [pr + [ construct_chat_dict("user",x)] for x in left_questions]

        #batch_left_ans = answer_batch_questions(left_questions,lpr,label,cache_fn,model_arch,50,verbose=verbose)

        batch_left_ans = answer_batch_questions_chat(left_questions,lpr,label,cache_fn,model_arch,50,verbose=verbose)

        print('left answer',batch_left_ans)

        right_questions = [node.key_right.replace('!!x!!',x) for x in node.r.answers[-1]]
        print('right questions',right_questions)

        #rpr = [pr + inst_funct(x) for x in right_questions]
        rpr = [pr + [ construct_chat_dict("user",x)] for x in right_questions]



        #batch_right_ans = answer_batch_questions(right_questions,rpr,label,cache_fn,model_arch,400,verbose=verbose)
        batch_right_ans = answer_batch_questions_chat(right_questions,rpr,label,cache_fn,model_arch,50,verbose=verbose)

        print('right answer',batch_right_ans)

        left = pd.DataFrame({'left':node.l.answers[-1],"key":batch_left_ans})

        right = pd.DataFrame({'right':node.r.answers[-1],"key":batch_right_ans})
        print(f"Left is: {left}")
        print(f"Right is: {right}")
        print(left.merge(right,on='key',how='inner'))
        ans = list (left.merge(right,on='key',how='inner')[node.filter_key])
        print('JOINED ANSWER',ans)
        node.answers = ans
        node.status = status

        return



    node_text_list = node.text
    print(node_text_list)
    adjusted_nodes_list = node.adjusted_nodes
    if verbose:
        print('Tree Nodes: ',(["_".join(x) for x in adjusted_nodes_list]))

    for adjusted_node in  adjusted_nodes_list:
        op = adjusted_node[0]

        if 'AGGREGATE_count_star()' in "_".join(adjusted_node) or 'AGGREGATE_OP_count' in "_".join(adjusted_node):
            try:

                ans = len(node.answers[-1] if len(adjusted_nodes_list)>1 else node.l.answers[-1])

            except:
                status = 'FAILED AGGREGATE OPERATION'
                ans=[]

            #temp_questions.append('COUNT')
            node.questions.append('COUNT')
            node.status = status
            #temp_unfilt_ans.append(ans)
            node.unfiltered_answers.append(ans)

            #temp_ans.append(ans)
            node.answers.append(ans)

            if verbose:
                print('Answer: ',ans)


        elif 'AGGREGATE_OP' in op:

            func = adjusted_node[1]
            #proj_ind = node_ind-1

            #if verbose:
            #    print('PROJ IND:',proj_ind)
            #    print('Answers: ',temp_ans[proj_ind])

            #temp_questions.append(func+'('+",".join(temp_ans[proj_ind])+')')
            node.questions.append(func+'('+",".join(node.answers[-1])+')')



            #prev_ans=[]

            try:
                #prev_ans = [x[:-1] if x[-1]=='.' else x for x in temp_ans[proj_ind]]

                prev_ans = node.l.ans[-1]
                prev_ans = [x[:-1] if x[-1]=='.' else x for x in prev_ans]

                if func!='count':
                    numer_ans = [replace_units(x) for x in prev_ans]
                    #print('Replacing UNITS:',numer_ans)
                    numer_ans = [re.sub("[^0-9.*]","",x) for x in numer_ans]
                    #print('Remove other text:',numer_ans)
                    numer_ans = [x for x in numer_ans if x]
                    numer_ans = [x[:-1] if x[-1]=='.' else x for x in numer_ans]
                    numer_ans = [x for x in numer_ans if x]
                    numer_ans = [eval(x) for x in numer_ans]
                    #print('Evaluate numbers:',numer_ans)
                else:
                    numer_ans = [x for x in numer_ans if x]
                #if verbose:
                    #print('Removing non-nuemrical: ',numer_ans)
                if func!='count':
                    numer_ans = [ float(x) for x in numer_ans]
                if verbose:
                    print('Numerical Parsing: ',numer_ans)
                ans = map_func(func)(numer_ans)
                node.status = 'FINISHED'

            except:
                node.status = 'FAILED AGGREGATE OPERATION'
                ans=[]

            #temp_unfilt_ans.append(ans)
            node.unfiltered_answers.append(ans)
            #temp_ans.append(ans)
            node.answers.append(ans)

            if verbose:
            #    print('Function used: ',map_func(func))
                print('Unfiltered answer:',ans)
                print('Filtered answer:',ans)
                print('Status: ',status)
        else:
            k = '_'.join(adjusted_node)
            if hasattr(node, 'filled_questions'):
              question = node.filled_question + "Don’t justify your answers. Don’t give information not asked in the CONTEXT INFORMATION."
            elif k in augmented_question_maps:
              question = augmented_question_maps[k] + "Don’t justify your answers. Don’t give information not asked in the CONTEXT INFORMATION."
              print(f"k exists! k value is {k}")
            else:
              question = prompt_create(dfTrain, k)['choices'][0]['message']['content'] + " Don’t justify your answers. Don’t give information not asked in the CONTEXT INFORMATION."
              # print(f"question is: {question['choices'][0]['message']['content']}")
            #question = node.filled_question if hasattr(node,'filled_question') else augmented_question_maps[k]
            #print(f"Converted Question: {question}")
            if verbose:
                print('OP: ',k)
                print('Q: ',question)

            # adjust max length generation
            if op == 'SEQ_SCAN':max_len = 400
            elif op=='FILTER': max_len=1 if 'turbo' not in model_arch else 2
            else: max_len = 50

            #proj_ind = -1

            if op=='AGGREGATE_PROJ':
                prev_ans = node.l.answers[-1]
            elif  op =='PROJECTION':
                prev_ans = node.l.answers[-1]
            else:
                prev_ans = node.l.answers[-1] if node.l and node.l.answers else []


            #     proj_ind = [ind for ind,x in enumerate(tree_nodes) if x[0]=='AGGREGATE_PROJ'][0]-1
            # elif  op=='PROJECTION':
            #     proj_ind = [ind for ind,x in enumerate(tree_nodes) if x[0]=='PROJECTION'][0]-1

            if '!!x!!' in question: model_questions = [question.replace('!!x!!',x) for x in prev_ans ]
            #elif '!!list!!' in question:
            #    prev_ans = temp_ans[proj_ind]
            #    model_questions = [question.replace('!!list!!',", ".join(prev_ans))]
            else: model_questions = [question]

            node.questions.append(model_questions)
            #pr= instr+few_shots+""
            #pr = [pr + inst_funct(x) for x in model_questions]

            pr=construct_message_dict(instr,few_shots)
            pr = [pr +[ construct_chat_dict("user",x)] for x in model_questions]




            #if verbose:    print('Prompt: ',pr)


            ans=[]
            if op == 'SEQ_SCAN':
                if k in cache:
                    ans = cache[k]
                else:
                    print('NOT IN CACHE')
                    old_pr = pr[0]
                    #if verbose:
                    #    print('RUNNING SEQUENTIAL SCANS...')
                    #    print(old_pr)

                    #response = completion_with_backoff(model=model_arch, prompt=old_pr, temperature=0,max_tokens= max_len)
                    #print(f"Old PR is: {old_pr}\n")
                    response = completion_with_backoff_chat(model=model_arch, messages=old_pr, temperature=0,max_tokens= max_len)

                    print(f"Completion with backoff chat response: {response}")

                    old_pr_key = json.dumps(old_pr) if isinstance(old_pr,list) else old_pr
                    cache[old_pr_key] = response

                    json.dump(cache,open(cache_fn,"w"),indent=2)

                    #old_ans = response['choices'][0]['text']
                    old_ans = response['choices'][0]['message']['content']
                    if verbose:
                        print(old_ans)
                    temp_unfilt_ans = []
                    temp_questions = []
                    ans = [add_more_seq_scan(temp_unfilt_ans,model_arch,temp_questions,old_pr,old_ans,max_tries=2,increase_threshold=5)]

                    node.unfiltered_answers.append(temp_unfilt_ans)
                    node.questions.append(temp_questions)

                    cache[k] = ans
                    json.dump(cache,open(cache_fn,"w"),indent=2)


            else:
                    batch_ans = answer_batch_questions_chat(model_questions,pr,label,cache_fn,model_arch,max_len,verbose=verbose)
                    ans.extend(batch_ans)


            node.unfiltered_answers.append(ans)
            if verbose:   print('Unfiltered Answer: ',ans)

            if op == 'SEQ_SCAN':
              if ans:
                  ans = ans[0]
                  ans = ans[:-1] if ans[-1]=='.' else ans
                  ans = ans.replace(' and ','')
                  ans = ans.split(',') #if ',' in ans else ans.split(' ')
                  ans = [x for x in ans if '.' not in x] #remove 'India.Yemen'
                  ans = list(set(ans))
                  ans =[x for x in ans if x]
              node.answers.append(ans)
              if verbose:
                  print('Final Answer: ',ans)

            elif op == 'FILTER':
                filtered_ans=[]
                if ans:
                    bool_index = [x.replace('.','').strip() for x in ans]
                    bool_index = [x=='Yes' for x in bool_index]
                    filtered_ans = list(compress(node.l.answers[-1], bool_index))
                if verbose:
                    print('Final Answer: ', filtered_ans)
                node.answers.append(filtered_ans)
                if not filtered_ans:
                    status = 'EMPTY'
                    if verbose:
                        print('EMPTY')
                    break

            elif op != 'AGGREGATE_OP' and op!='AGGREGATE_count_star()':
                  #ans = ans[0]
                  node.answers.append(ans)
                  if verbose:
                      print('Final Answer: ', ans)
            print('\n')
            node.status = status


def compute_tree(node,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=False):
    if node and node.text:
        print(f"Node Questions: {node.questions}")
        print(f"Node Answers: {node.answers}")
        compute_tree(node.l,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=verbose)
        compute_tree(node.r,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=verbose)
        compute_node(node,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=verbose)



# from QueryTree import *

import time

c5=None

def GPT_SPW_seq(model_arch,df,instr,few_shots,inst_funct,label,augmented_question_maps,query_plan_dict,verbose=False):
    global b
    # create file for answers
    json.dump([],open(label+".json","w"),indent=3)

    # check if there is cache data
    cache_fn=label+'_cache.json'
    mode = 'r' if cache_fn in os.listdir('.') else 'w'
    if verbose: print('Mode: ',mode)
    cache=json.load(open(cache_fn,'r')) if mode=='r' else dict()

    print("Checkpoint 1: Before the FOR loop\n")

    Questions = []
    Answers = []
    Unfiltered_Answers = []

    for index,row in df.iterrows():
        # get query
        query = row.Query

        # get duckdb con
        # con = run_db(db_files[row.Database])
        # con.execute("PRAGMA enable_profiling='query_tree';")
        # con.execute("PRAGMA explain_output='ALL';")

        #get logical execution plan
        if verbose:
            print(f"Query: {query}")
            print('\n')
        if query in query_plan_dict:
            root = query_plan_dict[query]
        else:
            try:
                con.execute("EXPLAIN "+query.replace('"',"'"))
                s = con.fetchall()[0][1].split('\n')
                if verbose:
                    print("\n".join(s))
                    print('\n')
                root = parse_query_tree(s)
            except Exception as e:
                print(e)
                print("comes into the exception, so continues")
                continue
        print("Out Of The If/Else Block")
        b=root
        #tree_nodes = []
        #get_tree_elements(root,tree_nodes)


        #invert tree nodes
        #tree_nodes = tree_nodes[::-1]



        #separate filter
        #tree_nodes = adjust_nodes(tree_nodes)
        tree_adjust_nodes(root)

        #if verbose:
        #    print_tree(root)

        # temp_ans=[]
        # temp_unfilt_ans=[]
        # temp_questions=[]

        compute_tree(root,model_arch,instr,few_shots,inst_funct,label,augmented_question_maps,verbose=verbose)
        tree_nodes,questions,answers,unfiltered_answers = get_snippet(root,[],[],[],[])

        Questions += questions[0]
        Unfiltered_Answers += unfiltered_answers[0]
        Answers += answers[0]

        snippet = {'Gold Question':row.Question,'Gold Answer':row.Answer,'Query':row.Query,
                   'Tree Nodes':tree_nodes,'LP Questions':questions,'LP Answers':answers,
                   'LP Unfiltered Answers':unfiltered_answers,'qqqqqqqqqqqq':root.status}



        log = json.load(open(label+".json","r"))
        log.append(snippet)
        json.dump(log,open(label+".json","w"),indent=3)
        #json.dump(cache,open(cache_fn,"w"),indent=2)
        time.sleep(3)
        print("===================================================================================")
    return Questions, Answers, Unfiltered_Answers

## **Selection**
Since using the complete Spider dataset would use up our tokens, we select a few queries that are not expensive

In [4]:
# sel_DF is the Selected DataFrame
sel_DF = df.iloc[12:13]

#sel_indices = np.r_[40, 46:48]
#sel_DF = df.iloc[sel_indices]

#50, 42, 41, 40, 38, 37, 34, 3

sel_DF = sel_DF.reset_index(drop=True)
for column in sel_DF.columns:
  print(sel_DF[column])

0    what is the average population of the us by state
Name: Question, dtype: object
0    [(4415590.666666667,)]
Name: Answer, dtype: object
0    geo
Name: Database, dtype: object
0    SELECT AVG ( population ) FROM state;
Name: Query, dtype: object
0    Spider
Name: Dataset, dtype: object
0    SPA
Name: Type, dtype: object
0    SAP
Name: Type2, dtype: object


## **Creating DFs**

In [5]:
import pandas as pd


create_DF = pd.DataFrame([
{
'Question': 'What are the names of all the continents?',
'Answer': ['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Australia', 'Antarctica'],
'Database': 'geo',
'Query': 'SELECT T1.name FROM continents as T1',
'Dataset': 'Spider',
'Type': 'SPA',
'Type2': 'SAP'
},
{
'Question': 'List all colors of the rainbow.',
'Answer': ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet'],
'Database': 'world_1',
'Query': 'SELECT T1.name FROM rainbow_colors as T1',
'Dataset': 'Spider',
'Type': 'SPA',
'Type2': 'SAP'
},
{
"Question": "What are the names of all the oceans?",
"Answer": ["Pacific Ocean", "Atlantic Ocean", "Indian Ocean", "Southern Ocean", "Arctic Ocean"],
"Database": "geo",
"Query": "SELECT T1.name FROM oceans as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
{
"Question": "List the twelve signs of the zodiac",
"Answer": ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"],
"Database": "astrology",
"Query": "SELECT T1.name FROM zodiac_signs as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
{
"Question": "List the planets of the solar system.",
"Answer": ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"],
"Database": "space",
"Query": "SELECT T1.name FROM planets as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
{
"Question": "List the twelve dwarfs from 'Snow White'.",
"Answer": ["Doc", "Grumpy", "Happy", "Sleepy", "Bashful", "Sneezy", "Dopey", "Humbert", "Grimm", "Bossy", "Biggy", "Smoky"],
"Database": "characters",
"Query": "SELECT T1.name FROM snow_white_dwarfs as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
# {
# "Question": "Name the five permanent members of the United Nations Security Council.",
# "Answer": ["China", "France", "Russia", "United Kingdom", "United States"],
# "Database": "UN",
# "Query": "SELECT T1.name FROM security_council as T1 WHERE T1.permanent_member = True",
# "Dataset": "Spider",
# "Type": "SPA",
# "Type2": "SAP"
# },
{
"Question": "List the seven deadly sins.",
"Answer": ["Lust", "Gluttony", "Greed", "Sloth", "Wrath", "Envy", "Pride"],
"Database": "sins",
"Query": "SELECT T1.name FROM deadly_sins as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
{
"Question": "Name five Nobel Prize categories.",
"Answer": ["Physics", "Chemistry", "Medicine", "Literature", "Peace"],
"Database": "nobel_prizes",
"Query": "SELECT T1.name FROM nobel_prize_categories as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
},
{
"Question": "Name all seven books in the Harry Potter series.",
"Answer": ["Harry Potter and the Philosopher's Stone", "Harry Potter and the Chamber of Secrets", "Harry Potter and the Prisoner of Azkaban", "Harry Potter and the Goblet of Fire", "Harry Potter and the Order of the Phoenix", "Harry Potter and the Half-Blood Prince", "Harry Potter and the Deathly Hallows"],
"Database": "harry_potter",
"Query": "SELECT T1.name FROM harry_potter_titles as T1",
"Dataset": "Spider",
"Type": "SPA",
"Type2": "SAP"
}
])

# **TREE BUILDING OF THE 50 QUERIES**

In [6]:
# tree_nodes = []

# db_files = {'geo':'data/geography-db.added-in-2020.sqlite',
#             'world_1':'spider/database/world_1/world_1.sqlite',
#             'flight_4':'spider/database/flight_4/flight_4.sqlite',
#             'flight_2':'spider/database/flight_2/flight_2.sqlite',
#             'singer':'spider/database/singer/singer.sqlite'
#             }

# for i, (db, q) in enumerate(zip(create_DF.Database, create_DF.Query)):
#     print(i)
#     print(create_DF.Question[i])
#     intermediate_tree_nodes = []
#     print(q)

#     con = run_db(db_files[db])
#     con.execute("PRAGMA enable_profiling='query_tree';")
#     con.execute("PRAGMA explain_output='ALL';")
#     con.execute("EXPLAIN " + q.replace('"', "'"))
#     s = con.fetchall()[0][1]
#     print(s)

#     try:
#         root = parse_query_tree(s.split('\n'))
#         get_tree_elements(root, intermediate_tree_nodes)
#         tree_nodes.append(intermediate_tree_nodes[::-1])
#         print('PARSED')
#     except:
#         print('NOT PARSED')

#     print('=======================================')

In [7]:
join_query_trees={}
tree_nodes = []


# EXTRA
# How many continents exist?
q = 'SELECT T1.name FROM continents as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','continents']
c1.op = 'SEQ_SCAN'
c1.args=['continents']

# c2 = Node()
# c2.text = ['PROJECTION','official_name']
# c2.op = 'PROJECTION'
# c2.args=['Official Name']
# c2.l = c1

join_query_trees[q] = c1

# List all colors of the rainbow
q = 'SELECT T1.name FROM rainbow_colors as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','rainbow_colors']
c1.op = 'SEQ_SCAN'
c1.args=['rainbow_colors']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1


# What are the names of all the oceans?
q = 'SELECT T1.name FROM oceans as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','oceans']
c1.op = 'SEQ_SCAN'
c1.args=['oceans']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM zodiac_signs as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','zodiac_signs']
c1.op = 'SEQ_SCAN'
c1.args=['zodiac_signs']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM planets as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','planets']
c1.op = 'SEQ_SCAN'
c1.args=['zodiac_signs']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM snow_white_dwarfs as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','snow_white_dwarfs']
c1.op = 'SEQ_SCAN'
c1.args=['snow_white_dwarfs']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM deadly_sins as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','deadly_sins']
c1.op = 'SEQ_SCAN'
c1.args=['deadly_sins']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM nobel_prize_categories as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','nobel_prize_categories']
c1.op = 'SEQ_SCAN'
c1.args=['nobel_prize_categories']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

#
q = 'SELECT T1.name FROM harry_potter_titles as T1'
c1 = Node()
c1.text = ['SEQ_SCAN','harry_potter_titles']
c1.op = 'SEQ_SCAN'
c1.args=['harry_potter_titles']

# c2 = Node()
# c2.text = ['PROJECTION','name']
# c2.op = 'PROJECTION'
# c2.args=['Name']
# c2.l = c1

join_query_trees[q] = c1

################################################################################



# 40
# Which language is the most popular in Aruba?
q = 'SELECT T2.Language FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.Name = "Aruba" ORDER BY Percentage DESC LIMIT 1'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           #[0.0]          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           LIMIT           │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │          ORDER_BY         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          ORDERS:          │
# │           #[0.1]          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          Language         │
# │         Percentage        │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │      (Name = 'Aruba')     │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘

# NOT PARSED
c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!?. Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Aruba')"]
c4.op = 'FILTER'
c4.args=["(Name = 'Aruba')"]

c5 = Node()
c5.text = ['PROJECTION','Language','Percentage']
c5.op = 'PROJECTION'
c5.args=['Language','Percentage']


c6 = Node()
c6.text = ['ORDER_BY','Percentage']
c6.op = 'ORDER_BY'
c6.args=['Percentage']


c7 = Node()
c7.text = ['PROJECTION','Language']
c7.op = 'PROJECTION'
c7.args=['Language']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
c6.l=c5
c7.l=c6






# =======================================
# 41
# what is the capital of states that have cities named durham
q = 'SELECT t2.capital FROM state AS t2 JOIN city AS t1 ON t2.state_name = t1.state_name WHERE t1.city_name = "durham";'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          capital          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │   (city_name = 'durham')  │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │ (state_name = state_name) │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││            city           │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','state']
c1.op = 'SEQ_SCAN'
c1.args=['state']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the state name of !!x!!? Answer briefly.'
c3.key_right = 'What is the state name of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(city_name = 'durham')"]
c4.op = 'FILTER'
c4.args=["(city_name = 'durham')"]
c4.filled_question = 'Is !!x!! the same as Durham?'
c5 = Node()
c5.text = ['PROJECTION','capital']
c5.op = 'PROJECTION'
c5.args=['capital']
c5.filled_question = 'What is the capital of state of !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5



# NOT PARSED
# =======================================
# 42
# What are the regions that use English or Dutch?
q = 'SELECT DISTINCT T1.Region FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T2.Language = "English" OR T2.Language = "Dutch"'
# ┌───────────────────────────┐
# │          DISTINCT         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │       "T1"."Region"       │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           Region          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │((Language = 'English') OR │
# │   (Language = 'Dutch'))   │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Language = 'English')","(Language = 'Dutch')"]
c4.op = 'FILTER'
c4.args=["(Language = 'English')","(Language = 'Dutch')"]

c5 = Node()
c5.text = ['PROJECTION','Region']
c5.op = 'PROJECTION'
c5.args=['Region']
c5.filled_question = 'What is the region that speaks !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5





# NOT PARSED
# =======================================
# 43
# What is the official language spoken in the country whose head of state is Beatrix?
q = 'SELECT T2.Language FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.HeadOfState = "Beatrix" AND T2.IsOfficial = "T"'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          Language         │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │ (HeadOfState = 'Beatrix') │
# │ (IsOfficial = CAST('T' AS │
# │          BOOLEAN))        │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'


c4 = Node()
c4.text = ['FILTER',"(HeadOfState = 'Beatrix')","(IsOfficial = CAST('T' AS BOOLEAN))"]
c4.op = 'FILTER'
c4.args=["(HeadOfState = 'Beatrix')","(IsOfficial = CAST('T' AS BOOLEAN))"]

c5 = Node()
c5.text = ['PROJECTION','Language']
c5.op = 'PROJECTION'
c5.args=['Language']
c5.filled_question = 'What is the language of !!x!!?'



c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q]=c5





# NOT PARSED
# =======================================
# 44
# what state has no rivers
# SELECT state_name FROM state WHERE state_name NOT IN ( SELECT traverse FROM river );
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │         state_name        │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │       (NOT SUBQUERY)      │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │            MARK           ├──────────────┐
# │   (state_name = #[8.0])   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││          traverse         │
# └───────────────────────────┘└─────────────┬─────────────┘
#                              ┌─────────────┴─────────────┐
#                              │          SEQ_SCAN         │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │           river           │
#                              └───────────────────────────┘






# NOT PARSED
# =======================================
# 45
# how many rivers do not traverse the state with the capital albany
# SELECT COUNT ( river_name ) FROM river WHERE traverse NOT IN ( SELECT state_name FROM state WHERE capital = "albany" );
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │     count(river_name)     │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         AGGREGATE         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │     count(river_name)     │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │       (NOT SUBQUERY)      │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │            MARK           ├──────────────┐
# │    (traverse = #[8.0])    │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           river           ││         state_name        │
# └───────────────────────────┘└─────────────┬─────────────┘
#                              ┌─────────────┴─────────────┐
#                              │           FILTER          │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │    (capital = 'albany')   │
#                              └─────────────┬─────────────┘
#                              ┌─────────────┴─────────────┐
#                              │          SEQ_SCAN         │
#                              │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
#                              │           state           │
#                              └───────────────────────────┘








# NOT PARSED
# =======================================
# 46
# What is the number of distinct continents where Chinese is spoken?
q ='SELECT COUNT( DISTINCT Continent) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T2.Language = "Chinese"'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │ count(DISTINCT Continent) │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         AGGREGATE         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │ count(DISTINCT Continent) │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │   (Language = 'Chinese')  │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Language = 'Chinese')"]
c4.op = 'FILTER'
c4.args=["(Language = 'Chinese')"]
c4.filled_question = 'Is !!x!! the same as Chinese?'

c5 = Node()
c5.text = ['AGGREGATE','count(Continent)']
c5.op = 'AGGREGATE'
c5.args=['count(Continent)']
c5.filled_question = 'What is the continent that speaks !!x!!?'

# c6 = Node()
# c6.text = ['PROJECTION','count(Continent)']
# c6.op = 'PROJECTION'
# c6.args=['count(Continent)']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
#c6.l=c5

join_query_trees[q] = c5






# NOT PARSED
# =======================================
# 47
# Which region is the city Kabul located in?
q = 'SELECT Region FROM country AS T1 JOIN city AS T2 ON T1.Code = T2.CountryCode WHERE T2.Name = "Kabul"'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           Region          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │      (Name = 'Kabul')     │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││            city           │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Kabul')"]
c4.op = 'FILTER'
c4.args=["(Name = 'Kabul')"]
c4.filled_question = 'Is !!x!! the same as Kabul?'
c5 = Node()
c5.text = ['PROJECTION','Region']
c5.op = 'PROJECTION'
c5.args=['Region']
c5.filled_question = 'What is the region of !!x!!?'

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4


join_query_trees[q] = c5








# NOT PARSED
# =======================================
# 48
# How many official languages does Afghanistan have?
q = 'SELECT COUNT(*) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.Name = "Afghanistan" AND IsOfficial = "T"'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │        count_star()       │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │         AGGREGATE         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │        count_star()       │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │   (Name = 'Afghanistan')  │
# │ (IsOfficial = CAST('T' AS │
# │          BOOLEAN))        │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │    (Code = CountryCode)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          country          ││      countrylanguage      │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','country']
c1.op = 'SEQ_SCAN'
c1.args=['country']

c2 = Node()
c2.text = ['SEQ_SCAN','countrylanguage']
c2.op = 'SEQ_SCAN'
c2.args=['countrylanguage']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the country code of !!x!!? Answer briefly.'
c3.key_right = 'What is the country code of the country that speaks !!x!!? Answer briefly.'
c3.filter_key='left'

c4 = Node()
c4.text = ['FILTER',"(Name = 'Afghanistan')","(IsOfficial = CAST('T' AS BOOLEAN))"]
c4.op = 'FILTER'
c4.args=["(Name = 'Afghanistan')","(IsOfficial = CAST('T' AS BOOLEAN))"]

c5 = Node()
c5.text = ['AGGREGATE','count_star()']
c5.op = 'AGGREGATE'
c5.args=['count_star()']

# c6 = Node()
# c6.text = ['PROJECTION','count_star()']
# c6.op = 'PROJECTION'
# c6.args=['count_star()']

c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4
#c6.l=c5

join_query_trees[q] = c5




# NOT PARSED
# =======================================
# 49
# which capitals are not major cities
q = 'SELECT t2.capital FROM state AS t2 JOIN city AS t1 ON t2.capital = t1.city_name WHERE t1.population <= 150000;'
# ┌───────────────────────────┐
# │         PROJECTION        │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │          capital          │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │           FILTER          │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │ (population <= CAST(150000│
# │         AS BIGINT))       │
# └─────────────┬─────────────┘
# ┌─────────────┴─────────────┐
# │      COMPARISON_JOIN      │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           INNER           ├──────────────┐
# │   (capital = city_name)   │              │
# └─────────────┬─────────────┘              │
# ┌─────────────┴─────────────┐┌─────────────┴─────────────┐
# │          SEQ_SCAN         ││          SEQ_SCAN         │
# │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
# │           state           ││            city           │
# └───────────────────────────┘└───────────────────────────┘

c1 = Node()
c1.text = ['SEQ_SCAN','state']
c1.op = 'SEQ_SCAN'
c1.args=['state']

c2 = Node()
c2.text = ['SEQ_SCAN','city']
c2.op = 'SEQ_SCAN'
c2.args=['city']

c3 = Node()
c3.text = ['JOIN','city']
c3.op = 'JOIN'
c3.key_left='What is the capital of !!x!!? Answer briefly.'
c3.key_right = 'What is the name of !!x!!? Answer briefly.'
c3.filter_key='right'

c4 = Node()
c4.text = ['FILTER',"(population <= CAST(150000 AS BIGINT))"]
c4.op = 'FILTER'
c4.args=["(population <= CAST(150000 AS BIGINT))"]

c5 = Node()
c5.text = ['PROJECTION','capital']
c5.op = 'PROJECTION'
c5.args=['capital']
c5.filled_question  = 'What is the capital of the state of !!x!!?'


c3.l=c1
c3.r = c2
c4.l=c3
c5.l=c4

join_query_trees[q] = c5


# NOT PARSED
# =======================================


In [8]:

question_maps = {'SEQ_SCAN_state': 'List some american states.',
 "FILTER_(state_name = 'new mexico')": 'Is !!x!! the same as "New Mexico"?',
 'PROJECTION_area': 'What is the area of !!x!!?',
 "FILTER_(state_name = 'california')": 'Is !!x!! the same as California?',
 'PROJECTION_population': 'What is the population of !!x!!?',
 'SEQ_SCAN_city': 'List some cities.',
 "FILTER_(state_name = 'texas')": 'Is !!x!! the same as Texas?',
 'PROJECTION_city_name': 'What is the name of !!x!!?',
 "FILTER_(capital = 'albany')": 'Is Albany the capital of !!x!!?',
 'SEQ_SCAN_lake': 'List some american lakes.',
 'PROJECTION_lake_name': 'What is the name of the lake !!x!!?',
 'FILTER_(area > CAST(750 AS DOUBLE) )': 'Is the area of !!x!! greater than 750 km2?',
 "FILTER_(state_name = 'michigan')": 'Is !!x!! in Michigan?',
 'FILTER_(lowest_elevation = 0)': 'Is the lowest elevation in !!x!! 0 meters?',
 'PROJECTION_highest_point': 'What is the highest point of !!x!!?',
 'PROJECTION_state_name': 'What is the name of the state of !!x!!?',
 'SEQ_SCAN_river': 'List some rivers in the US.',
 "FILTER_(traverse = 'idaho')": 'Does !!x!! traverse Idaho?',
 'AGGREGATE_PROJ_river_name': 'What is the name of the river !!x!!?',
 "FILTER_(traverse = 'illinois')": 'Does !!x!! traverse Illinois?',
 'PROJECTION_river_name': 'WHat is the name of !!x!!?',
 "FILTER_(city_name = 'springfield')": 'Is !!x!! the same as Springfield?',
 "FILTER_(city_name = 'boulder')": 'Is !!x!! the same as Boulder?',
 "FILTER_(state_name = 'delaware')": 'Is !!x!! in Delaware?',
 'PROJECTION_highest_elevation': 'What is the highest elevation of !!x!!?',
 "FILTER_(highest_point = 'guadalupe peak')": 'Is guadalupe peak the highest point of !!x!!?',
 'AGGREGATE_PROJ_highest_elevation': 'What is the highest elevation of !!x!!?',
 "FILTER_(river_name = 'rio grande')": 'Is !!x!! the same as "Rio Grande"?',
 'PROJECTION_length': 'What is the length of !!x!!?',
 "FILTER_(state_name = 'rhode island ')": 'Is !!x!! the same as "rhode island"?',
 'AGGREGATE_PROJ_capital': 'What is the capital of !!x!!?',
 'AGGREGATE_PROJ_city_name': 'What is the name of !!x!!?',
 'FILTER_(population > 150000)': 'Does !!x!! have a population greater than 150000?',
 "FILTER_(river_name = 'colorado')": 'Is !!x!! the same as Colorado?',
 "FILTER_(city_name = 'seattle')": 'Is !!x!! the same as Seattle?',
 "FILTER_(state_name = 'washington')": 'Is !!x!! in Washington?',
 'AGGREGATE_PROJ_population': 'What is the popualtion of !!x!!?',
 'AGGREGATE_PROJ_state_name': 'What is the name of the state of !!x!!?',
 'FILTER_(length > 750)': 'Is the length of !!x!! greater than 750 miles?',
 'PROJECTION_capital': 'What is the capital of !!x!!?',
 "FILTER_(state_name = 'kansas')": 'Is !!x!! in the state of Kanses?',
 "FILTER_(state_name = 'wisconsin')": 'Is !!x!! in the state of Wisconsin?',
 'AGGREGATE_PROJ_area': 'What is the area of !!x!!?',
 "FILTER_(state_name = 'wyoming')": 'Is !!x!! the same as Wyoming?',
 'PROJECTION_density': 'What is the density of !!x!!?',
 'PROJECTION_lowest_point': 'What is the lowest point of !!x!!?',
 'AGGREGATE_PROJ_length': 'What is the length of !!x!!?',
 'PROJECTION_traverse': 'What states does !!x!! traverse?',
 'SEQ_SCAN_mountain': 'List some mountains in the US.',
 "FILTER_(mountain_name = 'whitney')": 'Is !!x!! the same as "Whitney"?',
 "FILTER_(city_name = 'austin')": 'Is !!x!! the same as Austin?',
 "FILTER_(capital = 'salem')": 'Is Salem the capital of !!x!!?',
 "FILTER_(river_name = 'missouri')": 'Is !!x!! the same as "missouri"?',
 'AGGREGATE_PROJ_traverse': 'What are the states that !!x!! traverses?',
 "FILTER_(state_name = 'pennsylvania ')": 'Is !!x!! the same as pennsylvania? ',
 'PROJECTION_(CAST(population AS DOUBLE) / area)': 'What is the population of !!x!! divided by its area?',
 "FILTER_(traverse = 'ohio')": 'Does !!x!! traverse Ohio?',
 'AGGREGATE_PROJ_DISTINCT traverse': 'What are the states that !!x!! traverses?',
 "FILTER_(mountain_name = 'mckinley' )": 'Is !!x!! the same as Mckinley?',
 'PROJECTION_mountain_altitude': 'What is the altitude of !!x!!?',
 "FILTER_(state_name = 'alaska')": 'Is !!x!! in Alaska?',
 'PROJECTION_mountain_name': 'What is the name of !!x!!?',
 "FILTER_(state_name = 'massachusetts')": 'Is !!x!! the same as massachusetts?',
 'PROJECTION_country_name': 'What is the name of the country of !!x!!?',
 "FILTER_(capital = 'austin')": 'Is Austin the capital of !!x!!?',
 'SEQ_SCAN_country': 'List some countries.',
 'FILTER_(IndepYear > 1950)': 'Is the independence year of !!x!! greater than 1950?',
 'PROJECTION_Name': 'What is the name of !!x!!?',
 "FILTER_(GovernmentForm = 'Republic ')": 'Is the government form of !!x!! republic?',
 'AGGREGATE_count_star()': 'What is the count of the following list: !!list!!?',
 "FILTER_(Region = 'Caribbean')": 'Is !!x!! in the Caribbean?',
 'AGGREGATE_PROJ_SurfaceArea': 'What is the surface area of !!x!!?',
 "FILTER_(Name = 'Anguilla')": 'Is !!x!! Anguilla?',
 'PROJECTION_Continent': 'What is the continent of !!x!!?',
 "FILTER_(Name = 'Brazil')": 'Is !!x!! the same as Brazil?',
 'PROJECTION_Population': 'What is the population of !!x!!?',
 'PROJECTION_LifeExpectancy': 'What is the life expectancy of !!x!!?',
 "FILTER_(Name = 'Angola')": 'Is !!x!! Angola?',
 'PROJECTION_Region': 'What is the region of !!x!!?',
 "FILTER_(Region = 'Central Africa')": 'Is !!x!! in central Africa?',
 'AGGREGATE_PROJ_LifeExpectancy': 'What is the life expectancy of !!x!!?',
 "FILTER_(Continent = 'Asia')": 'Is !!x!! in Asia?',
 'AGGREGATE_PROJ_Population': 'What is the population of !!x!!?',
 'AGGREGATE_PROJ_GNP': 'What is the GNP of !!x!!?',
 "FILTER_(Continent = 'Africa')": 'Is !!x!! in Africa?',
 "FILTER_(District = 'Gelderland')": 'Is !!x!! in Gelderland district?',
 "FILTER_(GovernmentForm = 'US Territory')": 'Does !!x!! have US Territory as a form of government?',
 'SEQ_SCAN_countrylanguage': 'List some languages.',
 'AGGREGATE_PROJ_DISTINCT Language': 'What is the language of !!x!!?',
 'AGGREGATE_PROJ_DISTINCT GovernmentForm': 'What is the government form of !!x!!?',
 "FILTER_(Language != 'English')": 'Is !!x!! not English?',
 'PROJECTION_CountryCode': 'What is the alpha-3 country code of !!x!!?',
 'DISTINCT_countrylanguage._"CountryCode"': 'What is the alpha-3 country code of !!x!!?',
 'PROJECTION_SurfaceArea': 'What is the surface area of !!x!!?',
 "FILTER_(Continent = 'North America ')": 'Is !!x!! in North America?',
 'FILTER_(SurfaceArea > CAST(3000 AS DOUBLE))': 'Does !!x!! have a surface area greater than 3000?',
 'FILTER_(Population >= 160000)': 'Does !!x!! have a population greater than 160000?',
 'FILTER_(Population <= 900000)': 'Does !!x!! have a population less than 900000?',
 'SEQ_SCAN_airports': 'List some airports.',
 "FILTER_(city = 'Goroka')": 'Is !!x!! in Goroka?',
 'PROJECTION_name': 'What is the name of !!x!!?',
 "FILTER_(city = 'New York')": 'Is !!x!! in New York?',
 'PROJECTION_city': 'What is the city of !!x!!?',
 'PROJECTION_country': 'What is the country of !!x!!?',
 'PROJECTION_elevation': 'What is the elevation of !!x!!?',
 "FILTER_(country = 'Iceland')": 'Is !!x!! in Iceland?',
 'AGGREGATE_PROJ_elevation': 'What is the elevation of !!x!!?',
 'SEQ_SCAN_airlines': 'List some airlines.',
 "FILTER_(name ~~ 'Orbit%')": 'Does !!x!! start with "Orbit"?',
 'FILTER_(elevation >= -50)': 'Is the elevation of !!x!! less than or equal to -50?',
 'FILTER_(elevation <= 50)': 'Is the elevation of !!x!! greater than or equal to 50?',
 "FILTER_(country = 'Greenland')": 'Is !!x!! in Greenland?',
 'AGGREGATE_PROJ_DISTINCT city': 'What is the city of !!x!!?',
 "FILTER_(Airline = 'JetBlue Airways ')": 'Is !!x!! the same as "JetBlue Airways"?',
 'PROJECTION_Country': 'What is the country of !!x!!?',
 'PROJECTION_Abbreviation': 'What is the abbreviation of !!x!!?',
 "FILTER_(Country = 'USA')": 'Is !!x!! in the USA?',
 'PROJECTION_Airline': 'What is the airline of !!x!!?',
 'SEQ_SCAN_flights': 'List some flights.',
 "FILTER_(Abbreviation = 'UAL')": 'Is "UAL" the abbreviation of !!x!!?',
 'SEQ_SCAN_singer': 'List some singers.',
 "FILTER_(Citizenship != 'France')": 'Is the citizenship of !!x!! not French?',
 'PROJECTION_GNP': 'What is the GNP of !!x!!?',
 "FILTER_(Region = 'Southern Europe' )": 'Is !!x!! in Southern Europe?',
 'PROJECTION_Code': 'What is the alpha-3 country code of !!x!!?',
 'FILTER_(population <= CAST(150000 AS BIGINT))': 'Is the population of !!x!! less than or equal to 150000?',
 'FILTER_(elevation >= CAST(-50 AS BIGINT))':'Is the elevation of !!x!! greater than or equal to -50?',
 'FILTER_(elevation <= CAST(50 AS BIGINT))':'Is the elevation of !!x!! less than or equal to 50?',
 'FILTER_(Population >= CAST(160000 AS BIGINT))':'Is the population of !!x!! greater than or equal to 160000?',
 'FILTER_(Population <= CAST(900000 AS BIGINT))':'Is the population of !!x!! less than or equal to 900000?',
 'FILTER_(population > CAST(150000 AS BIGINT))':'Is the population of !!x!! greater than to 150000?',
 'FILTER_(IndepYear > CAST(1950 AS BIGINT))':'Is the independence year of  !!x!! grater than 1950?',
 'FILTER_(length > CAST(750 AS BIGINT))':'Is the length of !!x!! greater than 750 km?',
 "FILTER_(Language = 'Dutch')":'Is !!x!! the same as Dutch?',
 "FILTER_(IsOfficial = CAST('T' AS BOOLEAN))":'Does !!x!! have an official langauge?',
 "FILTER_(HeadOfState = 'Beatrix')":'Is Beatrix the head of state of !!x!!?',
 "FILTER_(Name = 'Afghanistan')":'Is !!x!! the same as Afghanistan?',
 "FILTER_(Language = 'English')":'Is !!x!! the same as English?'

 }


In [9]:
#join_query_trees = []
join_tree_nodes=[]

def get_join_tree_nodes(root,a=[]):
    if root:
        get_join_tree_nodes(root.l,a)
        get_join_tree_nodes(root.r,a)
        a.append(root.text)
    return a

for q in join_query_trees:
    a = get_join_tree_nodes(join_query_trees[q],[])
    join_tree_nodes.append(a)

flattened_tree_nodes = [xx for x in tree_nodes+join_tree_nodes for xx in x]
flattened_tree_nodes = adjust_nodes_old(flattened_tree_nodes)


augmented_question_maps = augment_questions(question_maps)


# **RUNNING CHATGPT MODEL - BASE 0**

In [10]:
inst_chatgpt = "You are a highly intelligent question answering bot. If I ask you a question that is rooted in truth, you will give you the answer. If I ask you a question that is nonsense, trickery, or has no clear answer, you will respond with 'Unknown'. You will answer concisely."
fewshot_chatgpt = [['What is human life expectancy in the United States?', '78.'],
 ['Who was president of the United States in 1955?', 'Dwight D. Eisenhower.'],
 ['Which party was founded by Gramsci?', 'Comunista.'],
 ['What is the capital of France?', 'Paris.'],
 ['What is a continent starting with letter O?', 'Oceania.'],
 ['Where were the 1992 Olympics held?', 'Barcelona.'],
 ['How many squigs are in a bonk?', 'Unknown']]


In [11]:
import random

def split_train_test(data, test_ratio):
    df = list(data.items())
    #df = [tup for tup in df in tup[0]]
    random.shuffle(df)
    test_size = int(len(df) * test_ratio)
    return [[f"{q}", f"{a}"] for q, a in df[test_size:]], [[f"{q}", f"{a}"] for q, a in df[:test_size]]

    #return [{"input": f"{q}", "output":f"{a}"} for q, a in df[test_size:]], [f"{q} :--> {a}" for q, a in df[:test_size]]

dfTrain, dfTest = split_train_test(question_maps, 0.1)

# Convert dataset into ChatGPT format
dfTrain = construct_message_dict(inst_chatgpt, dfTrain)

def prompt_create(dfTrain, q):
  sleep(25)
  print(f"Prompt created for '{q}'!")
  answer = openai.ChatCompletion.create(
          model='gpt-3.5-turbo',
          messages=dfTrain + [construct_chat_dict("user", q)],
          temperature=0,
          max_tokens=400
  )
  sleep(25)
  return answer

In [12]:
q1, a1, ua1 = GPT_SPW_seq(model_arch='gpt-3.5-turbo', df=create_DF, instr=inst_chatgpt, few_shots=fewshot_chatgpt, inst_funct=1, label='Chat-GPT3-FS', augmented_question_maps=augmented_question_maps, query_plan_dict=join_query_trees, verbose=True)

print(f"Questions: {q1}")
print(f"Answers: {a1}")
print(f"Unfiltered Answers: {ua1}")

Mode:  w
Checkpoint 1: Before the FOR loop

Query: SELECT T1.name FROM continents as T1


Out Of The If/Else Block
Node Questions: []
Node Answers: []
Mode:  w
['SEQ_SCAN', 'continents']
Tree Nodes:  ['SEQ_SCAN_continents']
Prompt created for 'SEQ_SCAN_continents'!


RateLimitError: ignored

In [ ]:
single_question_answers = [
    completion_with_backoff_chat(
        model='gpt-3.5-turbo',
        messages=construct_message_dict(inst_chatgpt, fewshot_chatgpt) + [construct_chat_dict("user", question)],
        temperature=0,
        max_tokens=400
    )['choices'][0]['message']['content']
    for question in sel_DF['Question']
]

chatgpt_final_result_df = pd.DataFrame(single_question_answers, columns=['Single Question Answer'])
chatgpt_final_result_df[['Question', 'Query Answer', 'Database', 'Query']] = sel_DF[['Question', 'Answer', 'Database', 'Query']]
chatgpt_final_result_df = chatgpt_final_result_df[['Question', 'Query Answer', 'Database', 'Query', 'Single Question Answer']]
chatgpt_final_result_df.to_csv('ChatGPT_50_Final.csv', index=False)

Questions: 1. Should I run all?
2. Give examples for improvements from each?
3.

List of all experiments


In he report, keep two columns of my prompts vs Muhammeds


# **EXPERIMENT 1**
To begin with, I consider augmented_question_maps as my dataset. I would like ChatGPT to infer similar relationships.

### **Fine Tuning**

In [ ]:
print(len(dfTrain))
print(len(dfTest))

251
13


In [ ]:

gen_prompt = []
for qa_pair in dfTest:
  pred_answer = prompt_create(dfTrain, qa_pair[0])["choices"][0]["message"]["content"]
  print(f"Predicted Answer: {pred_answer}")
  print(f"True Answer: {qa_pair[1]}")
  gen_prompt.append({'Predicted Answer': pred_answer, 'True Answer': qa_pair[1]})
  #sleep(21)

Prompt created for 'PROJECTION_GNP'!
Predicted Answer: What is the GNP of !!x!!?
True Answer: What is the GNP of !!x!!?
Prompt created for 'SEQ_SCAN_river'!
Predicted Answer: List some rivers.
True Answer: List some rivers in the US.
Prompt created for 'PROJECTION_Region'!
Predicted Answer: What is the region of !!x!!?
True Answer: What is the region of !!x!!?
Prompt created for 'PROJECTION_river_name'!
Predicted Answer: What is the name of the river of !!x!!?
True Answer: WHat is the name of !!x!!?
Prompt created for 'AGGREGATE_PROJ_state_name'!
Predicted Answer: What is the name of the state of !!x!!?
True Answer: What is the name of the state of !!x!!?
Prompt created for 'PROJECTION_Population'!
Predicted Answer: What is the population of !!x!!?
True Answer: What is the population of !!x!!?
Prompt created for 'AGGREGATE_PROJ_highest_elevation'!
Predicted Answer: What is the highest elevation of !!x!!?
True Answer: What is the highest elevation of !!x!!?
Prompt created for 'FILTER_(R

APIError: ignored

# **EXPERIMENT 2**

Here, we try the question_maps completely generated by ChatGPT.

### **Data Preparation**

In [ ]:
gpt_question_maps = {}
for q in question_maps.keys():
  a = prompt_create(dfTrain, q)["choices"][0]["message"]["content"]
  gpt_question_maps[q] = a
  sleep(21)
print(gpt_question_maps)

{'SEQ_SCAN_state': 'List some US states.'}


### **GPT Calls**

In [ ]:
GPT_SPW_seq(model_arch='gpt-3.5-turbo', df=sel_DF, instr=inst_chatgpt, few_shots=fewshot_chatgpt, inst_funct=1, label='Chat-GPT3-FS', augmented_question_maps=augment_questions(gpt_question_maps), query_plan_dict=join_query_trees, verbose=True)

# **EXPERIMENT 3**

In [ ]:

question_maps = {'SEQ_SCAN_state': 'List the American states.',
 "FILTER_(state_name = 'new mexico')": 'Is !!x!! the same as "New Mexico"?',
 'PROJECTION_area': 'What is the area of !!x!!?',
 "FILTER_(state_name = 'california')": 'Is !!x!! the same as California?',
 'PROJECTION_population': 'What is the population of !!x!!?',
 'SEQ_SCAN_city': 'List the cities.',
 "FILTER_(state_name = 'texas')": 'Is !!x!! the same as Texas?',
 'PROJECTION_city_name': 'What is the name of !!x!!?',
 "FILTER_(capital = 'albany')": 'Is Albany the capital of !!x!!?',
 'SEQ_SCAN_lake': 'List the American Lakes.',
 'PROJECTION_lake_name': 'What is the name of the lake !!x!!?',
 'FILTER_(area > CAST(750 AS DOUBLE) )': 'Is the area of !!x!! greater than 750 km2?',
 "FILTER_(state_name = 'michigan')": 'Is !!x!! in Michigan?',
 'FILTER_(lowest_elevation = 0)': 'Is the lowest elevation in !!x!! 0 meters?',
 'PROJECTION_highest_point': 'What is the highest point of !!x!!?',
 'PROJECTION_state_name': 'What is the name of the state of !!x!!?',
 'SEQ_SCAN_river': 'List the rivers in the US.',
 "FILTER_(traverse = 'idaho')": 'Does !!x!! traverse Idaho?',
 'AGGREGATE_PROJ_river_name': 'What is the name of the river !!x!!?',
 "FILTER_(traverse = 'illinois')": 'Does !!x!! traverse Illinois?',
 'PROJECTION_river_name': 'WHat is the name of !!x!!?',
 "FILTER_(city_name = 'springfield')": 'Is !!x!! the same as Springfield?',
 "FILTER_(city_name = 'boulder')": 'Is !!x!! the same as Boulder?',
 "FILTER_(state_name = 'delaware')": 'Is !!x!! in Delaware?',
 'PROJECTION_highest_elevation': 'What is the highest elevation of !!x!!?',
 "FILTER_(highest_point = 'guadalupe peak')": 'Is guadalupe peak the highest point of !!x!!?',
 'AGGREGATE_PROJ_highest_elevation': 'What is the highest elevation of !!x!!?',
 "FILTER_(river_name = 'rio grande')": 'Is !!x!! the same as "Rio Grande"?',
 'PROJECTION_length': 'What is the length of !!x!!?',
 "FILTER_(state_name = 'rhode island ')": 'Is !!x!! the same as "rhode island"?',
 'AGGREGATE_PROJ_capital': 'What is the capital of !!x!!?',
 'AGGREGATE_PROJ_city_name': 'What is the name of !!x!!?',
 'FILTER_(population > 150000)': 'Does !!x!! have a population greater than 150000?',
 "FILTER_(river_name = 'colorado')": 'Is !!x!! the same as Colorado?',
 "FILTER_(city_name = 'seattle')": 'Is !!x!! the same as Seattle?',
 "FILTER_(state_name = 'washington')": 'Is !!x!! in Washington?',
 'AGGREGATE_PROJ_population': 'What is the popualtion of !!x!!?',
 'AGGREGATE_PROJ_state_name': 'What is the name of the state of !!x!!?',
 'FILTER_(length > 750)': 'Is the length of !!x!! greater than 750 miles?',
 'PROJECTION_capital': 'What is the capital of !!x!!?',
 "FILTER_(state_name = 'kansas')": 'Is !!x!! in the state of Kanses?',
 "FILTER_(state_name = 'wisconsin')": 'Is !!x!! in the state of Wisconsin?',
 'AGGREGATE_PROJ_area': 'What is the area of !!x!!?',
 "FILTER_(state_name = 'wyoming')": 'Is !!x!! the same as Wyoming?',
 'PROJECTION_density': 'What is the density of !!x!!?',
 'PROJECTION_lowest_point': 'What is the lowest point of !!x!!?',
 'AGGREGATE_PROJ_length': 'What is the length of !!x!!?',
 'PROJECTION_traverse': 'What states does !!x!! traverse?',
 'SEQ_SCAN_mountain': 'List the mountains in the US.',
 "FILTER_(mountain_name = 'whitney')": 'Is !!x!! the same as "Whitney"?',
 "FILTER_(city_name = 'austin')": 'Is !!x!! the same as Austin?',
 "FILTER_(capital = 'salem')": 'Is Salem the capital of !!x!!?',
 "FILTER_(river_name = 'missouri')": 'Is !!x!! the same as "missouri"?',
 'AGGREGATE_PROJ_traverse': 'What are the states that !!x!! traverses?',
 "FILTER_(state_name = 'pennsylvania ')": 'Is !!x!! the same as pennsylvania? ',
 'PROJECTION_(CAST(population AS DOUBLE) / area)': 'What is the population of !!x!! divided by its area?',
 "FILTER_(traverse = 'ohio')": 'Does !!x!! traverse Ohio?',
 'AGGREGATE_PROJ_DISTINCT traverse': 'What are the states that !!x!! traverses?',
 "FILTER_(mountain_name = 'mckinley' )": 'Is !!x!! the same as Mckinley?',
 'PROJECTION_mountain_altitude': 'What is the altitude of !!x!!?',
 "FILTER_(state_name = 'alaska')": 'Is !!x!! in Alaska?',
 'PROJECTION_mountain_name': 'What is the name of !!x!!?',
 "FILTER_(state_name = 'massachusetts')": 'Is !!x!! the same as massachusetts?',
 'PROJECTION_country_name': 'What is the name of the country of !!x!!?',
 "FILTER_(capital = 'austin')": 'Is Austin the capital of !!x!!?',
 'SEQ_SCAN_country': 'List the countries.',
 'FILTER_(IndepYear > 1950)': 'Is the independence year of !!x!! greater than 1950?',
 'PROJECTION_Name': 'What is the name of !!x!!?',
 "FILTER_(GovernmentForm = 'Republic ')": 'Is the government form of !!x!! republic?',
 'AGGREGATE_count_star()': 'What is the count of the following list: !!list!!?',
 "FILTER_(Region = 'Caribbean')": 'Is !!x!! in the Caribbean?',
 'AGGREGATE_PROJ_SurfaceArea': 'What is the surface area of !!x!!?',
 "FILTER_(Name = 'Anguilla')": 'Is !!x!! Anguilla?',
 'PROJECTION_Continent': 'What is the continent of !!x!!?',
 "FILTER_(Name = 'Brazil')": 'Is !!x!! the same as Brazil?',
 'PROJECTION_Population': 'What is the population of !!x!!?',
 'PROJECTION_LifeExpectancy': 'What is the life expectancy of !!x!!?',
 "FILTER_(Name = 'Angola')": 'Is !!x!! Angola?',
 'PROJECTION_Region': 'What is the region of !!x!!?',
 "FILTER_(Region = 'Central Africa')": 'Is !!x!! in central Africa?',
 'AGGREGATE_PROJ_LifeExpectancy': 'What is the life expectancy of !!x!!?',
 "FILTER_(Continent = 'Asia')": 'Is !!x!! in Asia?',
 'AGGREGATE_PROJ_Population': 'What is the population of !!x!!?',
 'AGGREGATE_PROJ_GNP': 'What is the GNP of !!x!!?',
 "FILTER_(Continent = 'Africa')": 'Is !!x!! in Africa?',
 "FILTER_(District = 'Gelderland')": 'Is !!x!! in Gelderland district?',
 "FILTER_(GovernmentForm = 'US Territory')": 'Does !!x!! have US Territory as a form of government?',
 'SEQ_SCAN_countrylanguage': 'List the languages.',
 'AGGREGATE_PROJ_DISTINCT Language': 'What is the language of !!x!!?',
 'AGGREGATE_PROJ_DISTINCT GovernmentForm': 'What is the government form of !!x!!?',
 "FILTER_(Language != 'English')": 'Is !!x!! not English?',
 'PROJECTION_CountryCode': 'What is the alpha-3 country code of !!x!!?',
 'DISTINCT_countrylanguage._"CountryCode"': 'What is the alpha-3 country code of !!x!!?',
 'PROJECTION_SurfaceArea': 'What is the surface area of !!x!!?',
 "FILTER_(Continent = 'North America ')": 'Is !!x!! in North America?',
 'FILTER_(SurfaceArea > CAST(3000 AS DOUBLE))': 'Does !!x!! have a surface area greater than 3000?',
 'FILTER_(Population >= 160000)': 'Does !!x!! have a population greater than 160000?',
 'FILTER_(Population <= 900000)': 'Does !!x!! have a population less than 900000?',
 'SEQ_SCAN_airports': 'List the airports.',
 "FILTER_(city = 'Goroka')": 'Is !!x!! in Goroka?',
 'PROJECTION_name': 'What is the name of !!x!!?',
 "FILTER_(city = 'New York')": 'Is !!x!! in New York?',
 'PROJECTION_city': 'What is the city of !!x!!?',
 'PROJECTION_country': 'What is the country of !!x!!?',
 'PROJECTION_elevation': 'What is the elevation of !!x!!?',
 "FILTER_(country = 'Iceland')": 'Is !!x!! in Iceland?',
 'AGGREGATE_PROJ_elevation': 'What is the elevation of !!x!!?',
 'SEQ_SCAN_airlines': 'List the airlines.',
 "FILTER_(name ~~ 'Orbit%')": 'Does !!x!! start with "Orbit"?',
 'FILTER_(elevation >= -50)': 'Is the elevation of !!x!! less than or equal to -50?',
 'FILTER_(elevation <= 50)': 'Is the elevation of !!x!! greater than or equal to 50?',
 "FILTER_(country = 'Greenland')": 'Is !!x!! in Greenland?',
 'AGGREGATE_PROJ_DISTINCT city': 'What is the city of !!x!!?',
 "FILTER_(Airline = 'JetBlue Airways ')": 'Is !!x!! the same as "JetBlue Airways"?',
 'PROJECTION_Country': 'What is the country of !!x!!?',
 'PROJECTION_Abbreviation': 'What is the abbreviation of !!x!!?',
 "FILTER_(Country = 'USA')": 'Is !!x!! in the USA?',
 'PROJECTION_Airline': 'What is the airline of !!x!!?',
 'SEQ_SCAN_flights': 'List the flights.',
 "FILTER_(Abbreviation = 'UAL')": 'Is "UAL" the abbreviation of !!x!!?',
 'SEQ_SCAN_singer': 'List the singers.',
 "FILTER_(Citizenship != 'France')": 'Is the citizenship of !!x!! not French?',
 'PROJECTION_GNP': 'What is the GNP of !!x!!?',
 "FILTER_(Region = 'Southern Europe' )": 'Is !!x!! in Southern Europe?',
 'PROJECTION_Code': 'What is the alpha-3 country code of !!x!!?',
 'FILTER_(population <= CAST(150000 AS BIGINT))': 'Is the population of !!x!! less than or equal to 150000?',
 'FILTER_(elevation >= CAST(-50 AS BIGINT))':'Is the elevation of !!x!! greater than or equal to -50?',
 'FILTER_(elevation <= CAST(50 AS BIGINT))':'Is the elevation of !!x!! less than or equal to 50?',
 'FILTER_(Population >= CAST(160000 AS BIGINT))':'Is the population of !!x!! greater than or equal to 160000?',
 'FILTER_(Population <= CAST(900000 AS BIGINT))':'Is the population of !!x!! less than or equal to 900000?',
 'FILTER_(population > CAST(150000 AS BIGINT))':'Is the population of !!x!! greater than to 150000?',
 'FILTER_(IndepYear > CAST(1950 AS BIGINT))':'Is the independence year of  !!x!! grater than 1950?',
 'FILTER_(length > CAST(750 AS BIGINT))':'Is the length of !!x!! greater than 750 km?',
 "FILTER_(Language = 'Dutch')":'Is !!x!! the same as Dutch?',
 "FILTER_(IsOfficial = CAST('T' AS BOOLEAN))":'Does !!x!! have an official langauge?',
 "FILTER_(HeadOfState = 'Beatrix')":'Is Beatrix the head of state of !!x!!?',
 "FILTER_(Name = 'Afghanistan')":'Is !!x!! the same as Afghanistan?',
 "FILTER_(Language = 'English')":'Is !!x!! the same as English?'

 }


In [ ]:
inst_chatgpt = "You are a highly intelligent question answering bot. I would like you to be measured with your replies. Please respond ONLY with the answers and no explanation. Limit your answers as much as possible. If I ask you a question that is rooted in truth, you will give you the answer. If I ask you a question that is nonsense, trickery, or has no clear answer, you will respond with 'Unknown'."
fewshot_chatgpt = [['What is human life expectancy in the United States?', '78.'],
 ['Who was president of the United States in 1955?', 'Dwight D. Eisenhower.'],
 ['Which party was founded by Gramsci?', 'Comunista.'],
 ['What is the capital of France?', 'Paris.'],
 ['What is a continent starting with letter O?', 'Oceania.'],
 ['Where were the 1992 Olympics held?', 'Barcelona.'],
 ['How many squigs are in a bonk?', 'Unknown']]

def split_train_test(data, test_ratio):
    df = list(data.items())
    #df = [tup for tup in df in tup[0]]
    random.shuffle(df)
    test_size = int(len(df) * test_ratio)
    return [[f"{q}", f"{a}"] for q, a in df[test_size:]], [[f"{q}", f"{a}"] for q, a in df[:test_size]]

    #return [{"input": f"{q}", "output":f"{a}"} for q, a in df[test_size:]], [f"{q} :--> {a}" for q, a in df[:test_size]]

dfTrain, dfTest = split_train_test(question_maps, 0.1)

# Convert dataset into ChatGPT format
dfTrain = construct_message_dict(inst_chatgpt, dfTrain)

def prompt_create(dfTrain, q):
  sleep(25)
  print(f"Prompt created for '{q}'!")
  answer = openai.ChatCompletion.create(
          model='gpt-3.5-turbo',
          messages=dfTrain + [construct_chat_dict("user", q)],
          temperature=0,
          max_tokens=400
  )
  sleep(25)
  return answer

q2, a2, ua2 = GPT_SPW_seq(model_arch='gpt-3.5-turbo', df=create_DF, instr=inst_chatgpt, few_shots=fewshot_chatgpt, inst_funct=1, label='Chat-GPT3-FS', augmented_question_maps=augmented_question_maps, query_plan_dict=join_query_trees, verbose=True)

print(f"Questions: {q2}")
print(f"Answers: {a2}")
print(f"Unfiltered Answers: {ua2}")

Mode:  r
Checkpoint 1: Before the FOR loop

Query: SELECT T1.name FROM continents as T1


Out Of The If/Else Block
Node Questions: []
Node Answers: []
Mode:  r
['SEQ_SCAN', 'continents']
Tree Nodes:  ['SEQ_SCAN_continents']
Prompt created for 'SEQ_SCAN_continents'!
OP:  SEQ_SCAN_continents
Q:  List the continents. Don’t justify your answers. Don’t give information not asked in the CONTEXT INFORMATION.
Unfiltered Answer:  ['Africa, Antarctica, Asia, Europe, North America, Oceania, South America.Unknown.']
Final Answer:  [' Asia', ' Antarctica', ' Europe', ' North America', 'Africa', ' Oceania']


Query: SELECT T1.name FROM rainbow_colors as T1


Out Of The If/Else Block
Node Questions: []
Node Answers: []
Mode:  r
['SEQ_SCAN', 'rainbow_colors']
Tree Nodes:  ['SEQ_SCAN_rainbow_colors']
Prompt created for 'SEQ_SCAN_rainbow_colors'!
OP:  SEQ_SCAN_rainbow_colors
Q:  List the colors of the rainbow. Don’t justify your answers. Don’t give information not asked in the CONTEXT INFORMATION.
Unfil

# **RESULTS**

## **Experiment 1 Results**

In [ ]:
from rouge import Rouge

hyps1, refs1 = map(list, zip(*[[d['Predicted Answer'], d['True Answer']] for d in gen_prompt]))

rouge = Rouge()
scores = rouge.get_scores(hyps1, refs1, avg=True)

for method in scores.keys():
  print(method)
  for metric in scores[method].keys():
    print(f"{metric}: {scores[method][metric]}")
  print("\n")

rouge-1
r: 0.914918414918415
p: 0.9231601731601733
f: 0.9166935578650693


rouge-2
r: 0.823076923076923
p: 0.8044871794871794
f: 0.8091715930622724


rouge-l
r: 0.8951048951048951
p: 0.9033466533466534
f: 0.8968800380515496




## **Experiment 2 Results**

In [ ]:
def stringify(input_list):
    output_list = []
    for el in input_list:
      output_list.append(str(sorted(el)).lower())
    return output_list

In [ ]:
Q = ['List some continents.', 'List some colors of the rainbow.', 'List some oceans.', 'List some zodiac signs.', 'List some planets.', 'List some snow white dwarfs.', 'List some deadly sins.', 'List some Nobel Prize categories.', 'List some Harry Potter titles.']
RA = [['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Australia', 'Antarctica'], ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet'], ['Pacific Ocean', 'Atlantic Ocean', 'Indian Ocean', 'Southern Ocean', 'Arctic Ocean'], ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'], ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'], ['Doc', 'Grumpy', 'Happy', 'Sleepy', 'Bashful', 'Sneezy', 'Dopey', 'Humbert', 'Grimm', 'Bossy', 'Biggy', 'Smoky'], ['Lust', 'Gluttony', 'Greed', 'Sloth', 'Wrath', 'Envy', 'Pride'], ['Physics', 'Chemistry', 'Medicine', 'Literature', 'Peace'], ["Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows']]
UFA = [['Africa, Antarctica, Asia, Europe, North America, Australia/Oceania, South America.I apologize for the oversight. Here are some additional continents: \n- Zealandia (sometimes considered a submerged continent)\n- Afro-Eurasia (combination of Africa and Eurasia)\n- Americas (combination of North and South America)'], ['Red, orange, yellow, green, blue, indigo, violet.Unknown.'], ['Atlantic Ocean, Pacific Ocean, Indian Ocean, Southern Ocean, Arctic Ocean.There are only five recognized oceans: Atlantic Ocean, Pacific Ocean, Indian Ocean, Southern Ocean, and Arctic Ocean.'], ['Aries, Taurus, Gemini, Cancer, Leo, Virgo, Libra, Scorpio, Sagittarius, Capricorn, Aquarius, Pisces.Unknown.'], ['Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.Pluto (formerly considered a planet), and there are also numerous exoplanets discovered outside our solar system.'], ['Unknown.Unknown.'], ["Pride, greed, lust, envy, gluttony, wrath, and sloth.Those are the seven deadly sins recognized in Christian teachings. However, it's worth noting that different cultures and religions may have different interpretations or additional sins."], ['Physics, Chemistry, Medicine, Literature, Peace, Economic Sciences.Sorry, but I can only provide the six Nobel Prize categories that are currently recognized.'], ['"Harry Potter and the Philosopher\'s Stone"\n"Harry Potter and the Chamber of Secrets"\n"Harry Potter and the Prisoner of Azkaban"\n"Harry Potter and the Goblet of Fire"\n"Harry Potter and the Order of the Phoenix"\n"Harry Potter and the Half-Blood Prince"\n"Harry Potter and the Deathly Hallows"I apologize for the confusion, but there are only seven main Harry Potter books as listed above. There are additional companion books and screenplays related to the Harry Potter series, but they are not part of the main book series.']]
FA = [['Europe', 'Asia', 'Australia/Oceania', 'Africa', 'North America', 'Antarctica'], ['green', 'orange', 'blue', 'yellow', 'Red', 'indigo'], ['Indian Ocean', 'Atlantic Ocean', 'Southern Ocean', 'Pacific Ocean', 'Arctic Ocean'], ['Cancer', 'Aquarius', 'Sagittarius', 'Scorpio', 'Libra', 'Capricorn', 'Virgo', 'Leo', 'Aries', 'Taurus', 'Gemini'], ['Mercury', 'Saturn', 'Uranus', 'Jupiter', 'Venus', 'Mars', 'there are also numerous exoplanets discovered outside our solar system', ' Earth'], [], ['gluttony', 'Pride', "it's worth noting that different culturesreligions may have different interpretations or additional sins", 'envy', 'greed', 'wrath', 'lust'], ['Medicine', 'Physics', 'but I can only provide the six Nobel Prize categories that are currently recognized', 'Literature', 'Peace', 'Chemistry'], ['"Harry Potterthe Philosopher\'s Stone"\n"Harry Potterthe Chamber of Secrets"\n"Harry Potterthe Prisoner of Azkaban"\n"Harry Potterthe Goblet of Fire"\n"Harry Potterthe Order of the Phoenix"\n"Harry Potterthe Half-Blood Prince"\n"Harry Potterthe Deathly Hallows"I apologize for the confusion', ' but they are not part of the main book series']]


#hyps1, refs1 = map(list, zip(*[stringify(RA), stringify(UFA)]))
#print(hyps1)
rouge = Rouge()

for i in range(0, len(RA)):
  print(stringify(RA)[i])
  print(stringify(FA)[i])
  print(rouge.get_scores(stringify(RA)[i], stringify(FA)[i]))
  print("\n")


exp2_scores = rouge.get_scores(stringify(RA), stringify(FA), avg=True)

for method in exp2_scores.keys():
  print(method)
  for metric in exp2_scores[method].keys():
    print(f"{metric}: {exp2_scores[method][metric]}")
  print("\n")

['africa', 'antarctica', 'asia', 'australia', 'europe', 'north america', 'south america']
['africa', 'antarctica', 'asia', 'australia/oceania', 'europe', 'north america']
[{'rouge-1': {'r': 0.8571428571428571, 'p': 0.6666666666666666, 'f': 0.7499999950781251}, 'rouge-2': {'r': 0.5, 'p': 0.375, 'f': 0.4285714236734694}, 'rouge-l': {'r': 0.8571428571428571, 'p': 0.6666666666666666, 'f': 0.7499999950781251}}]


['blue', 'green', 'indigo', 'orange', 'red', 'violet', 'yellow']
['red', 'blue', 'green', 'indigo', 'orange', 'yellow']
[{'rouge-1': {'r': 0.6666666666666666, 'p': 0.5714285714285714, 'f': 0.6153846104142012}, 'rouge-2': {'r': 0.4, 'p': 0.3333333333333333, 'f': 0.36363635867768596}, 'rouge-l': {'r': 0.6666666666666666, 'p': 0.5714285714285714, 'f': 0.6153846104142012}}]


['arctic ocean', 'atlantic ocean', 'indian ocean', 'pacific ocean', 'southern ocean']
['arctic ocean', 'atlantic ocean', 'indian ocean', 'pacific ocean', 'southern ocean']
[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.

## **Experiment 3 Results**

In [ ]:
RA = [['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Australia', 'Antarctica'], ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet'], ['Pacific Ocean', 'Atlantic Ocean', 'Indian Ocean', 'Southern Ocean', 'Arctic Ocean'], ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'], ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'], ['Doc', 'Grumpy', 'Happy', 'Sleepy', 'Bashful', 'Sneezy', 'Dopey', 'Humbert', 'Grimm', 'Bossy', 'Biggy', 'Smoky'], ['Lust', 'Gluttony', 'Greed', 'Sloth', 'Wrath', 'Envy', 'Pride'], ['Physics', 'Chemistry', 'Medicine', 'Literature', 'Peace'], ["Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows']]
FA = [['Asia', 'Antarctica', 'Europe', 'North America', 'Africa', 'Oceania'], ['blue', 'green', 'indigo', 'orange', 'Red', 'yellow'], ['Atlantic', 'Pacific', 'Indian', 'Southern'], ['Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Gemini', 'Aries', 'Capricorn', 'Sagittarius', 'Taurus', 'Aquarius'], ['Mercury', 'Makemake', 'Mars', 'Earth', 'Saturn', 'Uranus', 'Venus', 'Haumea', 'Eris', 'Jupiter'], ['Sneezy', 'Bashful', 'Doc', 'Sleepy', 'Happy', 'Grumpy'], ['lust', 'envy', 'gluttony', 'greed', 'wrath', 'Pride'], ['I cannot provide more Nobel Prize categories as there are only six categories', 'Peace', 'Physics', 'Chemistry', 'Literature', 'Medicine'], ['Harry Potterthe Order of the Phoenix', "Harry Potterthe Philosopher's Stone", 'Harry Potterthe Prisoner of Azkaban', 'Harry Potterthe Chamber of Secrets', 'Harry Potterthe Half-Blood Prince', 'Harry Potterthe Goblet of Fire']]
FA2 = [['Asia', 'Antarctica', 'Europe', 'North America', 'Africa', 'Oceania'], ['blue', 'green', 'indigo', 'orange', 'Red', 'yellow'], ['Atlantic Ocean', 'Pacific Ocean', 'Indian Ocean', 'Southern Ocean'], ['Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Gemini', 'Aries', 'Capricorn', 'Sagittarius', 'Taurus', 'Aquarius'], ['Mercury', 'Makemake', 'Mars', 'Earth', 'Saturn', 'Uranus', 'Venus', 'Haumea', 'Eris', 'Jupiter'], ['Sneezy', 'Bashful', 'Doc', 'Sleepy', 'Happy', 'Grumpy'], ['lust', 'envy', 'gluttony', 'greed', 'wrath', 'Pride'], ['I cannot provide more Nobel Prize categories as there are only six categories', 'Peace', 'Physics', 'Chemistry', 'Literature', 'Medicine'], ['Harry Potterthe Order of the Phoenix', "Harry Potterthe Philosopher's Stone", 'Harry Potterthe Prisoner of Azkaban', 'Harry Potterthe Chamber of Secrets', 'Harry Potterthe Half-Blood Prince', 'Harry Potterthe Goblet of Fire']]
UFA = [['Africa, Antarctica, Asia, Europe, North America, Oceania, South America.Unknown.'], ['Red, orange, yellow, green, blue, indigo, violet.Unknown.'], ['Atlantic, Pacific, Indian, Southern, Arctic.Unknown.'], ['Aries, Taurus, Gemini, Cancer, Leo, Virgo, Libra, Scorpio, Sagittarius, Capricorn, Aquarius, Pisces.Unknown.'], ['Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.Pluto (formerly considered a planet), Eris, Haumea, Makemake.'], ['Doc, Grumpy, Happy, Sleepy, Bashful, Sneezy, Dopey.Unknown.'], ['Pride, greed, lust, envy, gluttony, wrath, sloth.Unknown.'], ['Physics, Chemistry, Medicine, Literature, Peace, Economic Sciences.Sorry, I cannot provide more Nobel Prize categories as there are only six categories.'], ["Harry Potter and the Philosopher's Stone, Harry Potter and the Chamber of Secrets, Harry Potter and the Prisoner of Azkaban, Harry Potter and the Goblet of Fire, Harry Potter and the Order of the Phoenix, Harry Potter and the Half-Blood Prince, Harry Potter and the Deathly Hallows.Unknown."]]

#print(stringify(RA))

rouge = Rouge()

for i in range(0, len(RA)):
  print(stringify(RA)[i])
  print(stringify(FA2)[i])
  print(rouge.get_scores(stringify(RA)[i], stringify(FA2)[i]))
  print("\n")


exp3_scores = rouge.get_scores(stringify(RA), stringify(FA2), avg=True)

for method in exp3_scores.keys():
  print(method)
  for metric in exp3_scores[method].keys():
    print(f"{metric}: {exp3_scores[method][metric]}")
  print("\n")


# for ra, fa, scores in zip(stringify(RA), stringify(FA), scores):
#   print(ra)
#   print(fa)
#   print(score)
#   print("\n")

['africa', 'antarctica', 'asia', 'australia', 'europe', 'north america', 'south america']
['africa', 'antarctica', 'asia', 'europe', 'north america', 'oceania']
[{'rouge-1': {'r': 0.8571428571428571, 'p': 0.6666666666666666, 'f': 0.7499999950781251}, 'rouge-2': {'r': 0.6666666666666666, 'p': 0.5, 'f': 0.5714285665306124}, 'rouge-l': {'r': 0.8571428571428571, 'p': 0.6666666666666666, 'f': 0.7499999950781251}}]


['blue', 'green', 'indigo', 'orange', 'red', 'violet', 'yellow']
['red', 'blue', 'green', 'indigo', 'orange', 'yellow']
[{'rouge-1': {'r': 0.6666666666666666, 'p': 0.5714285714285714, 'f': 0.6153846104142012}, 'rouge-2': {'r': 0.4, 'p': 0.3333333333333333, 'f': 0.36363635867768596}, 'rouge-l': {'r': 0.6666666666666666, 'p': 0.5714285714285714, 'f': 0.6153846104142012}}]


['arctic ocean', 'atlantic ocean', 'indian ocean', 'pacific ocean', 'southern ocean']
['atlantic ocean', 'indian ocean', 'pacific ocean', 'southern ocean']
[{'rouge-1': {'r': 0.8333333333333334, 'p': 0.71428571

# **Code Dump**

In [ ]:
import openai

# Set up your OpenAI API credentials
openai.api_key = 'YOUR_API_KEY'

# Define your training data
training_data = [
    {
        'question': 'What is the capital of France?',
        'answer': 'Paris'
    },
    {
        'question': 'What is the capital of Germany?',
        'answer': 'Berlin'
    },
    # Add more question-answer pairs as needed
]

# Prepare the training data in the prompt and continuation format
formatted_data = ''
for data in training_data:
    formatted_data += f'Q: {data["question"]}\nA: {data["answer"]}\n'

# Define the fine-tuning parameters
fine_tuning_params = {
    'model': 'gpt-3.5-turbo',
    'dataset': {
        'prompt': formatted_data,
        'completion': None
    },
    'max_tokens': 1000,
    'n_epochs': 3,
    'learning_rate': 1e-5,
    'batch_size': 4,
    'overwrite_dataset': True,
    'validation_split': 0.1,
    'validation_every_n_epochs': 1
}

# Start the fine-tuning process
response = openai.ChatCompletion.create(**fine_tuning_params)

# Get the fine-tuned model ID
model_id = response['id']

# Wait for the fine-tuning to complete
while True:
    model = openai.ChatCompletion.fetch(model_id)
    if model['status'] == 'ready':
        break

# Use the fine-tuned model for inference
def infer_relationships(question):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': question}
        ],
        max_tokens=50
    )
    return response.choices[0].message.content

# Test the model by providing a question
question = 'What is the capital of Italy?'
inferred_relationship = infer_relationships(question)
print(f"Inferred Relationship: {inferred_relationship}")


In [ ]:
#print(sel_DF)
print(chatgpt_final_result_df)

                                            Question  \
0  What is the number of distinct continents wher...   
1         Which region is the city Kabul located in?   

                       Query Answer Database  \
0                            [(4,)]  world_1   
1  [('Southern and Central Asia',)]  world_1   

                                               Query  \
0  SELECT COUNT( DISTINCT Continent) FROM country...   
1  SELECT Region FROM country AS T1 JOIN city AS ...   

                    Single Question Answer  
0                                     One.  
1  Kabul is located in the region of Asia.  


In [ ]:
galois_ans = json.load(open('Chat-GPT3-FS.json','r'))

In [ ]:
i=0


print(f'Question :{galois_ans[i]["Gold Question"]}\n\n')

print(f'Query :{galois_ans[i]["Query"]}\n\n')

print(f'Status :{galois_ans[i]["Status"]}\n\n')


tree_nodes = galois_ans[i]["Tree Nodes"]

for k,tn in enumerate(tree_nodes):
    print(f'Tree Node:{tn}\n\n')

    print(f'Tree Node Question:{galois_ans[i]["LP Questions"][k]}\n\n')

    print(f'Tree Node Unfiltered Answers:{galois_ans[i]["LP Unfiltered Answers"][k]}\n\n')

    print(f'Tree Node Answers:{galois_ans[i]["LP Answers"][k]}\n\n')

    print('===============================================================================')

IndexError: ignored

In [ ]:
results_df = pd.read_csv('ChatGPT 50 Queries - Remove non-parsed(3).csv')
final_sample_df = pd.read_csv('Final_50_Sample.csv')
type_df = final_sample_df[['Question','Type2']]
results_df = results_df.merge(type_df,how='left',on='Question')


FileNotFoundError: ignored

In [ ]:
def get_final_type(x):
    if 'J' in x: return 'Join'
    if 'A' in x: return 'Agg'
    return 'Sel'

In [ ]:
results_df['Final Type']=results_df.Type2.apply(lambda x: get_final_type(x))
results_df.columns


NameError: ignored

In [ ]:
#rompt = "I want you to help me translate some snippets into a format that I define by example:\n" + "\n".join(dfTrain)

#print(prompt)

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=dfTrain,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.7,
    prompt_loss_weight=0.0,  # Use 0.0 for no influence on prompt style
    metadata={"language": "en"}  # Adjust metadata as needed
)


# Fine-tune the model
# response = openai.Completion.create(
#     engine='text-davinci-003',  # Use an appropriate engine for fine-tuning
#     prompt=prompt,
#     max_tokens=3500,  # Adjust as needed
#     n=1,  # Generate a single response
#     stop=None,  # Stop condition if required
#     temperature=0,  # Control randomness of the output
#     top_p=1.0,  # Control the diversity of the output
#     frequency_penalty=0.8,  # Adjust if too repetitive
#     presence_penalty=0.1,  # Adjust if too verbose
# )

model_id = response['model']
print(response['choices'])


model_file_path = "trained_model.txt"
with open(model_file_path, "w") as f:
    f.write(response["model"])

# Load the trained model for further testing or conversations
with open(model_file_path, "r") as f:
    trained_model = f.read()
